<a href="https://colab.research.google.com/github/Dedeepyamaddi/intel-unnati/blob/main/Image_Sharpening_Using_Knowledge_Distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install fpdf


  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=4ab8212ef439f6b13776c988a40248b56833d5b0099669629147d71d7e34be4d
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [ ]:
import os
import json
import numpy as np
from PIL import Image, UnidentifiedImageError
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from skimage.metrics import structural_similarity as ssim
from fpdf import FPDF
import shutil
from IPython.display import FileLink, display

# -------- Image Preprocessing ----------
def down_upscale_image(img_path, scale=0.75, resize_to=256):
    try:
        img = Image.open(img_path).convert('RGB')
    except (UnidentifiedImageError, OSError):
        raise ValueError(f"Image '{img_path}' is unreadable or corrupted.")
    img = img.resize((resize_to, resize_to), Image.BICUBIC)
    w, h = img.size
    img_down = img.resize((int(w * scale), int(h * scale)), Image.BICUBIC)
    img_up = img_down.resize((w, h), Image.BICUBIC)
    return img, img_up

# -------- Dataset ----------
class SharpnessDataset(Dataset):
    def __init__(self, image_dir, scale=0.75, resize_to=256):
        self.image_dir = image_dir
        self.scale = scale
        self.resize_to = resize_to
        self.images = [f for f in os.listdir(image_dir)
                       if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        self.transform = transforms.ToTensor()

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        high_res, degraded = down_upscale_image(img_path, self.scale, self.resize_to)
        return self.transform(degraded), self.transform(high_res), self.images[idx]

# -------- Model ----------
class LightDnCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(64, 3, 3, padding=1)
        )

    def forward(self, x):
        return x - self.layers(x)

# -------- Training ----------
def train_student(teacher, student, dataloader, device, epochs=10, alpha=0.7):
    teacher.eval()
    student.train()
    optimizer = torch.optim.Adam(student.parameters(), lr=1e-3)
    loss_fn = nn.MSELoss()
    logs = {"loss_per_epoch": []}

    for epoch in range(epochs):
        epoch_loss = 0.0
        for degraded, high_res, _ in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            degraded, high_res = degraded.to(device), high_res.to(device)
            with torch.no_grad():
                teacher_output = teacher(degraded)
            student_output = student(degraded)
            loss_gt = loss_fn(student_output, high_res)
            loss_teacher = loss_fn(student_output, teacher_output)
            loss = alpha * loss_teacher + (1 - alpha) * loss_gt
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        avg_loss = epoch_loss / len(dataloader)
        logs["loss_per_epoch"].append(avg_loss)
        print(f"Epoch {epoch+1} - Loss: {avg_loss:.6f}")
    return logs

# -------- Evaluation & Saving ----------
def evaluate_and_save(model, dataloader, device, save_folder='enhanced_output', originals_folder='originals', input_folder='images'):
    model.eval()
    os.makedirs(save_folder, exist_ok=True)
    os.makedirs(originals_folder, exist_ok=True)
    total_ssim = 0.0
    count = 0
    mos_feedback = {}

    with torch.no_grad():
        for degraded, high_res, filenames in tqdm(dataloader, desc="Evaluating"):
            degraded = degraded.to(device)
            output = model(degraded).cpu().numpy()
            target = high_res.numpy()

            for i in range(len(output)):
                out_img = np.moveaxis(output[i], 0, -1)
                tgt_img = np.moveaxis(target[i], 0, -1)
                out_img = np.clip(out_img, 0, 1)
                out_uint8 = (out_img * 255).astype(np.uint8)

                score = ssim(out_img, tgt_img, channel_axis=-1, data_range=1.0)
                total_ssim += score
                count += 1
                filename = filenames[i]
                Image.fromarray(out_uint8).save(os.path.join(save_folder, f'sharp_{filename}'))
                shutil.copy2(os.path.join(input_folder, filename), os.path.join(originals_folder, filename))
                mos_feedback[filename] = round(float(score * 100), 2)

    avg_ssim = total_ssim / count
    print(f"✅ Average SSIM: {avg_ssim * 100:.2f}%")

    with open("mos_feedback.json", "w") as f:
        json.dump(mos_feedback, f, indent=2)

    return avg_ssim

# -------- PDF Report ----------
def generate_pdf_report(avg_ssim, training_logs, pdf_path="report.pdf"):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=16)
    pdf.cell(200, 10, txt="Image Sharpness Report", ln=True, align='C')
    pdf.ln(10)
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt=f"Average SSIM: {avg_ssim * 100:.2f}%", ln=True)
    pdf.ln(5)
    pdf.cell(200, 10, txt="Training Loss per Epoch:", ln=True)
    for i, loss in enumerate(training_logs["loss_per_epoch"], 1):
        pdf.cell(200, 8, txt=f"Epoch {i}: {loss:.6f}", ln=True)
    pdf.output(pdf_path)
    print(f"✅ PDF report saved as '{pdf_path}'")

    with open("training.json", "w") as f:
        json.dump(training_logs, f, indent=2)
    print("✅ Training log saved as 'training.json'")

# -------- Main Pipeline --------
if __name__ == "__main__":
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    input_folder = "images"
    if not os.path.exists(input_folder) or len(os.listdir(input_folder)) == 0:
        raise FileNotFoundError(f"❌ No images found in '{input_folder}'. Please add your dataset.")

    teacher = LightDnCNN().to(device)
    student = LightDnCNN().to(device)

    dataset = SharpnessDataset(input_folder, scale=0.75, resize_to=256)
    dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=2)

    print("➡️ Training student model...")
    logs = train_student(teacher, student, dataloader, device, epochs=20)

    print("➡️ Evaluating and saving results...")
    avg_ssim = evaluate_and_save(student, dataloader, device)

    print("➡️ Generating PDF report...")
    generate_pdf_report(avg_ssim, logs)

    torch.save(student.state_dict(), 'student_model.pth')
    print("✅ Model saved as 'student_model.pth'")

    print("\n--- DONE ---")
    print(f"Average SSIM: {avg_ssim * 100:.2f}%")
    print("Enhanced images: enhanced_output/")
    print("Originals: originals/")


➡️ Training student model...


Epoch 1/20: 100%|██████████| 5/5 [00:13<00:00,  2.61s/it]


Epoch 1 - Loss: 0.002520


Epoch 2/20: 100%|██████████| 5/5 [00:13<00:00,  2.69s/it]


Epoch 2 - Loss: 0.000990


...

Epoch 19 - Loss: 0.000611


Epoch 20/20: 100%|██████████| 5/5 [00:12<00:00,  2.48s/it]


Epoch 20 - Loss: 0.000611
➡️ Evaluating and saving results...


Evaluating: 100%|██████████| 5/5 [00:03<00:00,  1.64it/s]

✅ Average SSIM: 94.08%
➡️ Generating PDF report...
✅ PDF report saved as 'report.pdf'
✅ Training log saved as 'training.json'
✅ Model saved as 'student_model.pth'

--- DONE ---
Average SSIM: 94.08%
Enhanced images: enhanced_output/
Originals: originals/
